In [2]:
import pandas as pd

In [3]:
accer_df2000=pd.read_csv("data/accre-jobs-2020.csv", nrows=2000 )

In [4]:
accer_df2000.head()

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,cn1531
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,cn1441
2,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,0:0,COMPLETED,cn1464
3,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24,production,0:0,COMPLETED,cn1473
4,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24,production,0:0,COMPLETED,cn1440


In [6]:
accer_df2000.tail()

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST
1995,17023702_397,portabella,cinnamon,4096Mn,213.09M,08:00:00,01:34:42,1,1,production,0:0,COMPLETED,cn1329
1996,17023702_398,portabella,cinnamon,4096Mn,213.09M,08:00:00,01:30:07,1,1,production,0:0,COMPLETED,cn1329
1997,17023702_399,portabella,cinnamon,4096Mn,212.91M,08:00:00,01:22:29,1,1,production,0:0,COMPLETED,cn1326
1998,17023702_400,portabella,cinnamon,4096Mn,211.71M,08:00:00,01:20:04,1,1,production,0:0,COMPLETED,cn1326
1999,17023702_401,portabella,cinnamon,4096Mn,212.33M,08:00:00,01:13:56,1,1,production,0:0,COMPLETED,cn1326


In [8]:
accer_df2000.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   JOBID      2000 non-null   object
 1   ACCOUNT    2000 non-null   object
 2   USER       2000 non-null   object
 3   REQMEM     2000 non-null   object
 4   USEDMEM    2000 non-null   object
 5   REQTIME    2000 non-null   object
 6   USEDTIME   2000 non-null   object
 7   NODES      2000 non-null   int64 
 8   CPUS       2000 non-null   int64 
 9   PARTITION  2000 non-null   object
 10  EXITCODE   2000 non-null   object
 11  STATE      2000 non-null   object
 12  NODELIST   2000 non-null   object
dtypes: int64(2), object(11)
memory usage: 203.2+ KB


Data Question 2: The Advanced Computing Center for Research and Education

In this data question, you will be analyzing data on jobs run on ACCRE's hardware. You will need to do some data cleaning and preparation and then explore and analyze to answer the following questions.



Data cleaning / formatting

We really only want to look at the "production" partition so rows with other partitions should be removed

Job time is in a format of either d-hh:mm:ss or hh:mm:ss, it needs to be converted to total seconds

Only successful jobs with "0:0" exit codes should be considered for memory use analysis

Memory is reported in terms of Megabytes per node (Mc) or Megabytes per core (Mc), this needs to be uniformly converted to Megabytes per core by dividing by the number of cores per node in a job.

In [20]:
accer_df2000['PARTITION'].value_counts()

production        1990
pascal               4
cgw-capra1           2
cgw-cqs1             2
turing               1
cgw-dougherty1       1
Name: PARTITION, dtype: int64

In [13]:
accer_production=accer_df2000[accer_df2000['PARTITION']=="production"]

In [19]:
accer_production['PARTITION'].value_counts()

production    1990
Name: PARTITION, dtype: int64

In [23]:
accer_production


,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,cn1531
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,cn1441
2,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,0:0,COMPLETED,cn1464
3,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24,production,0:0,COMPLETED,cn1473
4,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24,production,0:0,COMPLETED,cn1440
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,17023702_397,portabella,cinnamon,4096Mn,213.09M,08:00:00,01:34:42,1,1,production,0:0,COMPLETED,cn1329
1996,17023702_398,portabella,cinnamon,4096Mn,213.09M,08:00:00,01:30:07,1,1,production,0:0,COMPLETED,cn1329
1997,17023702_399,portabella,cinnamon,4096Mn,212.91M,08:00:00,01:22:29,1,1,production,0:0,COMPLETED,cn1326
1998,17023702_400,portabella,cinnamon,4096Mn,211.71M,08:00:00,01:20:04,1,1,production,0:0,COMPLETED,cn1326


In [25]:
new_used_time=accer_production['USEDTIME'].str.split("-",n=1, expand=True)

In [22]:
new_used_time

,0,1
0,13,18:00:28
1,12,14:50:56
2,13,18:00:20
3,12,03:50:32
4,13,18:00:03
...,...,...
1995,01:34:42,None
1996,01:30:07,None
1997,01:22:29,None
1998,01:20:04,None
